# Activity 14: Logistic Regression 

In [1]:
import os, glob
import numpy as np
import numpy.random as rand
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import offsetbox as obox
from mpl_toolkits.mplot3d import Axes3D
from sklearn import model_selection as slms, preprocessing as spre
from skimage import measure as meas, feature as feat
import cv2 as cv
from tqdm import tnrange, tqdm_notebook as tqdm
from jupyterthemes import jtplot


C:\Users\MaryChrisGo\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\MaryChrisGo\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\MaryChrisGo\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
traindir = 'C://Users//MaryChrisGo//Documents//1st Sem AY 2019-2020//App Physics 186//Act_14//banana'

In [3]:

class MCPerceptron:
    def __init__(self,
                 n_samples,
                 n_features,
                 epochs,
                 activation='step',
                 loss='sse',
                 lr=0.01,
                 eps=1e-6,
                 weight_initializer='random_uniform',
                 use_bias=False,
                 bias_initializer='ones'):
        rand.seed(401)
        self.n_samples = n_samples
        self.n_features = n_features
        self.epochs = epochs
        self.activation = activation
        self.loss = loss
        self.lr = lr
        self.eps = eps
        self.weight_initializer = weight_initializer
        self.use_bias = use_bias
        self.bias_initializer = bias_initializer
        
        if self.weight_initializer == 'random_uniform':
            if use_bias:
                self.W = rand.random(self.n_features + 1)
            else:
                self.W = rand.random(self.n_features)
        elif self.weight_initializer == 'random_normal':
            if use_bias:
                self.W = rand.normal((0, 1, (self.n_features + 1)))
            else:
                self.W = rand.normal((0, 1, (self.n_features)))
        else:
            raise NotImplementedError
            
        if self.use_bias:
            if self.bias_initializer == 'ones':
                self.b = np.ones(self.n_samples)
            else:
                raise NotImplementedError

        if self.activation == 'step':
            self.g = lambda a: 1 if a >= 0 else -1
        elif self.activation == 'sigmoid':
            self.g = lambda a: 1/(1 + np.exp(-a))
        elif self.activation == 'softmax':
            self.g = lambda a: np.exp(a)/np.sum(np.exp(a))
        else:
            raise NotImplementedError
            
        if self.loss == 'sse':
            self.l = lambda y_true, y_pred: (y_true - y_pred)**2
        else:
            raise NotImplementedError
    
    def train(self, x, y):
        self.graph_res = []
        x = np.column_stack((self.b, x))
        for count in tnrange(self.epochs, desc='Epoch'):
            res = 0
            for i in range(self.n_samples):
                a = x[i].T.dot(self.W)
                z = self.g(a)
                dW = self.lr * (y[i] - z) * x[i]
                self.W += dW
                res += self.l(z, y[i])
            res /= self.n_samples
            self.graph_res.append(res)
            if res < self.eps:
                break
        self.last_epoch = count + 1
                 
    def predict(self, X, y=None, metrics=None):
        if self.use_bias:
            z = np.zeros(X.shape[0] + 1)
            X = np.column_stack((np.ones(len(X)), X))
        else:
            z = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            a = X[i].dot(self.W)
            z[i] = self.g(a)
        if metrics is not None and 'acc' in metrics and y is not None:
            acc = 0.
            for i in range(len(y)):
                if np.round(z[i]).astype(int) == y[i]:
                    acc += 1
            acc /= len(y)
            self.metrics = dict({'acc': acc})
        self.preds = z
        return z
    
    def get_weights(self):
        return self.W
    
    def get_biases(self):
        return self.x.T[0]

    def get_line_params(self):
        W = self.W
        A, B, C = W[1], W[2], -W[0]
        m = -A/B
        b = C/B
        return m, b
    
    def save_weights(self, filename):
        np.save(filename, self.W)
        
    def load_weights(self, filename):
        self.W = np.load(filename)


def getChannelMeans(x):
    return np.array([np.mean(x[:,:,i]) for i in range(3)])


In [4]:
unripes = []
ripes = []

files = glob.glob(traindir + 'banana_unripe/*.jfif')
for i,f in enumerate(files):
    if i == 0:
        unripes = np.zeros((len(files)), '<U256')
    unripes[i] = f
    
files = glob.glob(traindir + 'banana_ripe/*.jfif')
for i,f in enumerate(files):
    if i == 0:
        ripes = np.zeros((len(files)), '<U256')
    ripes[i] = f


In [5]:
unripes = unripes[:len(ripes)]
banana = np.concatenate((unripes, ripes))
classes = np.concatenate((np.tile(0, len(unripes)), np.tile(1, len(ripes))))
x_train, x_test, y_train, y_test = slms.train_test_split(banana, classes,
                                                         test_size=0.2, 
                                                         shuffle=True)

In [7]:
def getFeature(X):
    for i,x in enumerate(X):
        if i == 0:
            features = np.zeros((len(X), 3))
        img = cv.imread(x)
        img = cv.cvtColor(img, cv.COLOR_BGR2Lab)/255
        features[i] = getChannelMeans(img)
    return features


In [8]:
features = getFeature(x_train)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
scatter = ax.scatter(features[:,1], features[:,2], 
                     marker='o', c=y_train, cmap='coolwarm')
legend = ax.legend(*scatter.legend_elements(), 
                   title='classes')
ax.add_artist(legend)
ax.set_xlabel('$a^*$')
ax.set_ylabel('$b^*$')
# plt.savefig('Lab_extract.png', dpi=300, bbox_inches='tight')
plt.show()

UnboundLocalError: local variable 'features' referenced before assignment